<a href="https://colab.research.google.com/github/nse-yu/colab-ML/blob/main/human_faces_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import os
import cv2
import glob
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
faces_info = pd.read_json("/content/drive/MyDrive/data/json/via_project_22Jun2022_13h14m_json.json")

> jsonファイルはデータフレームで受け取る

In [3]:
print(type(faces_info))

<class 'pandas.core.frame.DataFrame'>


> 確認

In [36]:
keys = faces_info.keys()
#print(keys)

#各エントリはタプルで与えられる
#各タプルは２要素構成で、0にエントリ名、1にすべてのデータが格納
for entry in faces_info.iterrows(): #filename, size, regions, file_attributes
  print(type(entry[1]),entry[0])

for key in faces_info.keys():
  print("----------------------------------")
  print(faces_info[key]) #Seriesオブジェクト単位

paths = [faces_info[key]["filename"] for key in faces_info.keys()]
print(paths)

<class 'pandas.core.series.Series'> filename
<class 'pandas.core.series.Series'> size
<class 'pandas.core.series.Series'> regions
<class 'pandas.core.series.Series'> file_attributes
----------------------------------
filename                                      Aaron_Eckhart_0001.jpg
size                                                            6855
regions            [{'shape_attributes': {'name': 'ellipse', 'cx'...
file_attributes                                                   {}
Name: Aaron_Eckhart_0001.jpg6855, dtype: object
----------------------------------
filename                                        Aaron_Guiel_0001.jpg
size                                                            9631
regions            [{'shape_attributes': {'name': 'ellipse', 'cx'...
file_attributes                                                   {}
Name: Aaron_Guiel_0001.jpg9631, dtype: object
----------------------------------
filename                                    Aaron_Patterson_0001.jp